In [ ]:
!pip install "opencv-python-headless<4.3"
!pip install cellpose

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 MB 12.5 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 2.0.5 requires opencv-python-headless>=4.9.0.80, but you have opencv-python-headless 3.4.18.65 which is incompatible.
albucore 0.0.23 requires opencv-python-headless>=4.9.0.80, but you have opencv-python-headless 3.4.18.65 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.9/225.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

In [ ]:
# Uninstall conflicting packages
!pip uninstall -y numpy opencv-python-headless cellpose

# Install compatible numpy version
!pip install numpy==1.24.3

# Install compatible opencv version
!pip install opencv-python-headless==4.8.0.74

# Reinstall cellpose
!pip install cellpose

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: opencv-python-headless 3.4.18.65
Uninstalling opencv-python-headless-3.4.18.65:
  Successfully uninstalled opencv-python-headless-3.4.18.65
Found existing installation: cellpose 3.1.1.1
Uninstalling cellpose-3.1.1.1:
  Successfully uninstalled cellpose-3.1.1.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 99.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 2.0.5 requires opencv-python-headless>=4.9.0.80, which is not installed.
albucore 0.0.23 requires opencv-python-headless>=4.9.0.80, which is not installed.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.3 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.3 which is incomp

Reason for being yanked: deprecated, use 4.8.0.76
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 11.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
albumentations 2.0.5 requires opencv-python-headless>=4.9.0.80, but you have opencv-python-headless 4.8.0.74 which is incompatible.
albucore 0.0.23 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
albucore 0.0.23 requires opencv-python-headless>=4.9.0.80, but you have opencv-python-headless 4.8.0.74 which is incompatible.
  Using cached cellpose-3.1.1.1-py3-none-any.whl.metadata (22 kB)
Using cached cellpose-3.1.1.1-py3-none-any.whl (225 kB)


In [ ]:
!nvcc --version
!nvidia-smi

import os, shutil
import numpy as np
import matplotlib.pyplot as plt
from cellpose import core, utils, io, models, metrics
from glob import glob

use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Wed Apr  2 08:47:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8       

In [ ]:
import numpy as np
import tifffile
from cellpose import models
import os
import glob
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define input and output paths
input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/1.4Pa-x20/Nuclei/tophat'
output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Nuclei_raw'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Define Cellpose parameters
model_type = 'cyto3'  # Using cyto3 model for nuclei
channels = [0, 0]     # Single channel mode (no nuclear channel needed since entire image is nuclei)
diameter = 30         # Approximate size of nuclei
cellprob_threshold = 0.0  # Threshold for cell probability
flow_threshold = 0.4      # Threshold for flows
resample = False      # Don't resample dynamics
do_3D = False         # 2D processing
stitch_threshold = 0  # No stitching needed for 2D

# Create Cellpose model
print("Creating Cellpose model...")
CP = models.CellposeModel(model_type=model_type, gpu=True)

# Find all .tif files in the input directory
input_files = glob.glob(os.path.join(input_dir, '*.tif'))
print(f"Found {len(input_files)} .tif files to process")

# Process each file
for input_file in input_files:
    # Get base filename for output
    filename = os.path.basename(input_file)
    base_name = os.path.splitext(filename)[0]
    output_file = os.path.join(output_dir, f"{base_name}_mask.tif")

    print(f"Processing: {filename}")

    # Load the image
    img = tifffile.imread(input_file)
    print(f"  Image shape: {img.shape}")

    # For 2D single-channel images, add dimensions to match what Cellpose expects
    if len(img.shape) == 2:
        # This is a 2D single-channel image (H, W)
        image = img[np.newaxis, ..., np.newaxis]  # Add z and channel dimensions (z, y, x, c)
    else:
        raise ValueError(f"Unexpected image shape: {img.shape}. Expected 2D image with shape (H, W).")

    # Run segmentation
    print(f"  Running segmentation with diameter={diameter}, cellprob_threshold={cellprob_threshold}, flow_threshold={flow_threshold}")
    masks, flows, _ = CP.eval(image,
                              channels=channels,
                              diameter=diameter,
                              resample=resample,
                              cellprob_threshold=cellprob_threshold,
                              flow_threshold=flow_threshold,
                              do_3D=do_3D,
                              stitch_threshold=stitch_threshold)

    # Process the masks
    masks = masks.squeeze()  # Remove any extra dimensions
    print(f"  Segmentation complete. Found {len(np.unique(masks))-1} objects")

    # Save the masks
    tifffile.imwrite(output_file, masks.astype(np.uint32))
    print(f"  Saved masks to {output_file}")

print("All processing complete!")

Mounted at /content/drive
Creating Cellpose model...


100%|██████████| 25.3M/25.3M [00:00<00:00, 30.4MB/s]


Found 34 .tif files to process
Processing: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Nuclei_regional_tophat.tif
  Image shape: (1024, 1024)
  Running segmentation with diameter=30, cellprob_threshold=0.0, flow_threshold=0.4
  Segmentation complete. Found 347 objects
  Saved masks to /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Nuclei_raw/denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Nuclei_regional_tophat_mask.tif
Processing: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq002_Nuclei_regional_tophat.tif
  Image shape: (1024, 1024)
  Running segmentation with diameter=30, cellprob_threshold=0.0, flow_threshold=0.4
  Segmentation complete. Found 290 objects
  Saved masks to /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Nuclei_raw/denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq002_Nuclei_regional_tophat_mask.tif
Processing: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq003_Nuclei_regional_tophat.tif
  Image shape: (1024,

In [4]:
import numpy as np
import tifffile
import os
import glob
from skimage import measure
import pandas as pd
from tqdm import tqdm  # For progress bar (optional, install with pip if needed)

# Define paths - update these to match your file structure
mask_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Nuclei_raw'
original_img_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/1.4Pa-x20/Nuclei/tophat'
output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Nuclei_filtered'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Define filtering criteria
MAX_INTENSITY_THRESHOLD = 50300
MEAN_INTENSITY_THRESHOLD = 30000
MIN_AREA_THRESHOLD = 100

# Find all mask files
mask_files = glob.glob(os.path.join(mask_dir, '*_mask.tif'))
print(f"Found {len(mask_files)} mask files to process")

# Initialize counters and lists for reporting
total_nuclei = 0
removed_nuclei = 0
filtered_stats = []

# Process each mask file
for mask_file in tqdm(mask_files, desc="Processing masks"):
    base_name = os.path.basename(mask_file).replace('_mask.tif', '')

    # Find corresponding original image for intensity measurements
    original_img_file = os.path.join(original_img_dir, f"{base_name}.tif")

    # Load mask
    mask = tifffile.imread(mask_file)

    # Only proceed if we have the original image for intensity measurements
    if not os.path.exists(original_img_file):
        print(f"Warning: Could not find original image for {base_name}. Skipping.")
        continue

    # Load original image
    original_img = tifffile.imread(original_img_file)

    # Get labeled regions
    regions = measure.regionprops(mask)
    total_nuclei += len(regions)

    # Create a new mask that will only contain the objects that pass our criteria
    filtered_mask = np.zeros_like(mask)

    # Process each object in the mask
    for region in regions:
        # Skip background (label 0)
        if region.label == 0:
            continue

        # Create a binary mask for this specific region
        object_mask = (mask == region.label)

        # Calculate area
        area = region.area

        # Calculate intensity statistics within this mask
        intensities = original_img[object_mask]
        mean_intensity = np.mean(intensities)
        max_intensity = np.max(intensities)

        # Check if the nucleus should be filtered out
        should_remove = (max_intensity > MAX_INTENSITY_THRESHOLD or
                         mean_intensity > MEAN_INTENSITY_THRESHOLD or
                         area < MIN_AREA_THRESHOLD)

        if should_remove:
            removed_nuclei += 1
            # Store information about the removed nucleus
            filtered_stats.append({
                'file': base_name,
                'nucleus_id': region.label,
                'area': area,
                'mean_intensity': mean_intensity,
                'max_intensity': max_intensity,
                'reason': ('max_intensity' if max_intensity > MAX_INTENSITY_THRESHOLD else
                          ('mean_intensity' if mean_intensity > MEAN_INTENSITY_THRESHOLD else 'area'))
            })
        else:
            # Keep this nucleus in the filtered mask
            filtered_mask[object_mask] = region.label

    # Save the filtered mask
    output_file = os.path.join(output_dir, f"{base_name}_filtered_mask.tif")
    tifffile.imwrite(output_file, filtered_mask)

    print(f"Processed {base_name}: {np.max(filtered_mask)} nuclei retained out of {len(regions)}")

# Calculate and display summary statistics
removal_percentage = (removed_nuclei / total_nuclei * 100) if total_nuclei > 0 else 0
print(f"\nFiltering complete!")
print(f"Total nuclei: {total_nuclei}")
print(f"Removed nuclei: {removed_nuclei} ({removal_percentage:.2f}%)")
print(f"Remaining nuclei: {total_nuclei - removed_nuclei} ({100 - removal_percentage:.2f}%)")

# Save filtering statistics to CSV
if filtered_stats:
    stats_df = pd.DataFrame(filtered_stats)

    # Count removals by reason
    reason_counts = stats_df['reason'].value_counts()
    print("\nRemoval reasons:")
    for reason, count in reason_counts.items():
        print(f"  {reason}: {count} nuclei ({count/removed_nuclei*100:.2f}%)")


Found 34 mask files to process


Processing masks:   3%|▎         | 1/34 [00:00<00:14,  2.24it/s]

Processed denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Nuclei_regional_tophat: 347 nuclei retained out of 347


Processing masks:   6%|▌         | 2/34 [00:01<00:18,  1.78it/s]

Processed denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq002_Nuclei_regional_tophat: 290 nuclei retained out of 290


...

Processed denoised_1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq016_Nuclei_regional_tophat: 250 nuclei retained out of 250


Processing masks:  97%|█████████▋| 33/34 [00:15<00:00,  2.19it/s]

Processed denoised_1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq017_Nuclei_regional_tophat: 277 nuclei retained out of 277


Processing masks: 100%|██████████| 34/34 [00:15<00:00,  2.13it/s]

Processed denoised_1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq018_Nuclei_regional_tophat: 320 nuclei retained out of 320

Filtering complete!
Total nuclei: 10318
Removed nuclei: 142 (1.38%)
Remaining nuclei: 10176 (98.62%)

Removal reasons:
  max_intensity: 125 nuclei (88.03%)
  area: 17 nuclei (11.97%)
